In [1]:
# library
import pandas as pd
import numpy as np

!pip install pmdarima
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from pmdarima.arima import ADFTest
import statsmodels.api as sm

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import math

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# IMPORT DATA FROM DATABASE

In [2]:
# IMPORT FROM XAMPP LOCALHOST
import pymysql
mysql_connection = pymysql.connect(host='localhost',
                    user='root',
                    password='',
                    db='csv_db 6',
                    charset='utf8',
                    cursorclass=pymysql.cursors.DictCursor)
                    
sql1 = "SELECT * FROM data_fertecon_2019_dst"
sql2 = "SELECT * FROM data_historis_usd_idr"
sql3 = "SELECT * FROM data_mop_2005_dst"

df_bench_dap = pd.read_sql(sql1, mysql_connection)
df_bench_kcl = pd.read_sql(sql3, mysql_connection)
df_usd_idr = pd.read_sql(sql2, mysql_connection)

In [53]:
# IMPORT FROM SQL
import pyodbc
server = '' 
database = '' 
username = '' 
password = ''
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

query = """
SELECT 
PENGADAAN_ME5A_R1.BANFN,Pengadaan_ME5A_R1.BNFPO,
Pengadaan_ME2N.EBELN,Pengadaan_ME2N.EBELP, 
Pengadaan_ME2N.BEDAT, Pengadaan_ME2N.EKORG, Pengadaan_ME2N.EKGRP,
Pengadaan_ME2N.EMATN, Pengadaan_ME2N.TXZ01,
Pengadaan_ME2N.NETPR AS UNITPRICE, Pengadaan_ME2N.CURRENCY_RATE, Pengadaan_ME2N.WAERS,
Pengadaan_ME2N.MENGE AS QTY
FROM Pengadaan_ME2N
LEFT JOIN Pengadaan_ME5A_R1
ON Pengadaan_ME2N.BANFN = Pengadaan_ME5A_R1.BANFN AND Pengadaan_ME2N.BNFPO = Pengadaan_ME5A_R1.BNFPO
WHERE (Pengadaan_ME2N.BANFN IS NOT NULL) AND (Pengadaan_ME5A_R1.BADAT IS NOT NULL) AND
(Pengadaan_ME2N.EMATN = '000000000001000017'OR Pengadaan_ME2N.EMATN = '000000000003000499' 
OR Pengadaan_ME2N.EMATN = '000000000003000266' OR Pengadaan_ME2N.EMATN = '000000000003000508'
OR Pengadaan_ME2N.EMATN = '000000000003000539'
OR Pengadaan_ME2N.EMATN = '000000000003000018'OR Pengadaan_ME2N.EMATN = '000000000003000072' 
OR Pengadaan_ME2N.EMATN = '000000000003000048' OR Pengadaan_ME2N.EMATN = '000000000003000019')

ORDER BY PENGADAAN_ME5A_R1.BANFN,Pengadaan_ME5A_R1.BNFPO,Pengadaan_ME2N.EBELN
"""

df_all = pd.read_sql_query(query, cnxn)

In [54]:
df_all

,BANFN,BNFPO,EBELN,EBELP,BEDAT,EKORG,EKGRP,EMATN,TXZ01,UNITPRICE,CURRENCY_RATE,WAERS,QTY
0,2100001642,00010,5000001438,00010,20210920,A000,A01,000000000003000019,KALIUM CHLORIDE:KCL;FLAKE,591.00,14.23301,USD,5000.000
1,2200000473,00010,5100000823,00010,20160413,B000,B01,000000000003000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.00,13.12300,USD,25000.000
2,2200000475,00010,5100000837,00010,20160414,B000,B01,000000000003000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.00,13.23800,USD,25000.000
3,2200001384,00010,5100001812,00010,20160427,B000,B01,000000000003000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.00,13.21500,USD,25000.000
4,2200001937,00010,5100002059,00010,20160504,B000,B01,000000000001000017,DAP:(NH4)2HPO4;GRANULAR;18%N 46%P2O5,344.00,13.24600,USD,13500.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,2600033074,00010,5500027840,00010,20211203,F000,F01,000000000003000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.00,14.35301,USD,4800.010
434,2600033090,00010,5500027838,00010,20211203,F000,F01,000000000003000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.00,14.35301,USD,1831.937
435,2600033091,00010,5500027837,00010,20211203,F000,F01,000000000003000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.00,14.35301,USD,4700.500
436,2600033520,00010,5500028311,00010,20220125,F000,F01,000000000003000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.00,14.34701,USD,4850.380


# BUILD MODEL

In [101]:
DAPKCL_EMATN = ['1000017','3000499', '3000539', '3000266', '3000018','3000072','3000048','3000019']


def pred_using_arima_for_dapkcl(DAPKCL_EMATN):
    
    df_final_all_ematn = pd.DataFrame()
    for j in DAPKCL_EMATN:

        # read data
        df_proc_1 = df_all.copy()
        df_proc_1

        # benerin ematn (ilangin 0 nya)
        df_proc_1['EMATN'] = df_proc_1['EMATN'].astype(str)
        df_proc_1['EMATN'] = df_proc_1['EMATN'].str[-7:]

        # pick ematn
        etn = j
        df_proc_1 = df_proc_1[df_proc_1['EMATN'] == etn].copy()

        # benerin bedat
        df_proc_1['BEDAT'] = df_proc_1['BEDAT'].astype(str)
        df_proc_1['BEDAT'] = df_proc_1['BEDAT'].str[6:8] + "/" + df_proc_1['BEDAT'].str[4:6] + "/" + df_proc_1['BEDAT'].str[:4]
        df_proc_1['BEDAT'] = pd.to_datetime(df_proc_1['BEDAT'])

        # benerin currency
        df_proc_1['CURRENCY_RATE'] = df_proc_1['CURRENCY_RATE'].astype(float) * 1000
        df_proc_1['UNITPRICE'] = df_proc_1['UNITPRICE'].astype(float)
        df_proc_1.loc[df_proc_1["WAERS"] == "USD", "UNITPRICE"] = df_proc_1['UNITPRICE']*df_proc_1['CURRENCY_RATE']
        df_proc_1 = df_proc_1[df_proc_1['EMATN'] == etn].loc[:,['BEDAT','UNITPRICE']].sort_values('BEDAT')
        df_proc_1 = df_proc_1.rename(columns = {'BEDAT' : 'date'})

        # per-week mode
        df_proc_2 = df_proc_1.groupby(pd.Grouper(key='date', axis=0, 
                                freq='W')).apply(lambda x: x.mode())
        df_proc_2 = df_proc_2.drop(columns=('date')).reset_index().drop(columns=('level_1')).groupby(pd.Grouper(key='date', axis=0, 
                                freq='W')).mean()

        # interpolate mengisi data yang nan/kosong
        df_proc_2 = df_proc_2.interpolate(method='linear', axis=0)

        #=============================================
        # DAP
        if j in ['1000017','3000499', '3000539', '3000266']:
            # atur usd df
            df_usd_idr_1 = df_usd_idr.loc[:,['Tanggal','Terakhir']]
            df_usd_idr_1['Tanggal'] = pd.to_datetime(df_usd_idr['Tanggal'])
            df_usd_idr_1['Terakhir'] = df_usd_idr_1['Terakhir'].str[:-2].str.replace('.','').astype(float)
            df_usd_idr_1_for_corr = df_usd_idr_1.groupby(pd.Grouper(key='Tanggal', axis=0, 
                                    freq='M')).mean().to_period('M').reset_index().rename(columns={'Tanggal':'date'})

            # atur idr to usd
            df_for_corr = df_proc_2.reset_index().groupby(pd.Grouper(key='date', axis=0, 
                                    freq='M')).mean().to_period('M').reset_index()

            df_for_corr_tousd = df_for_corr.merge(df_usd_idr_1_for_corr, on='date', how='left')
            df_for_corr_tousd['UNITPRICE_USD'] = df_for_corr_tousd['UNITPRICE'] / df_for_corr_tousd['Terakhir']
            df_for_corr_tousd['date'] = df_for_corr_tousd['date'].astype(str).str[:4] + 'M' + df_for_corr_tousd['date'].astype(str).str[5:7]
            df_for_corr_tousd.drop(columns=['UNITPRICE','Terakhir'],inplace=True)

            # CORRELATION
            # get 5 name good corr
            name_country_list = df_for_corr_tousd.merge(df_bench_dap, on='date', how='left').dropna().corr()['UNITPRICE_USD'].sort_values(ascending=False).index[1:6].to_list()

            # make dataframe from all good correlation with data
            df_country_list = df_bench_dap.loc[:,[name_country_list[0]]]
            df_country_list['date'] = pd.to_datetime(df_bench_dap['date'].str[:4] + '-' + df_bench_dap['date'].str[5:])
            df_country_list = df_country_list.set_index('date').to_period('M').reset_index()

            # merge two tables sap & bench
            sap_bench_gab_df = df_country_list.merge(df_proc_2.reset_index().groupby(pd.Grouper(key='date', axis=0, 
                                    freq='M')).mean().to_period('M').reset_index(), on='date', how='left').dropna()

            sap_bench_gab_df = sap_bench_gab_df.merge(df_usd_idr_1_for_corr, on='date', how='left')
            sap_bench_gab_df['UNITPRICE_USD'] = sap_bench_gab_df['UNITPRICE'] / sap_bench_gab_df['Terakhir']
            sap_bench_gab_df.drop(columns=['UNITPRICE','Terakhir'],inplace = True)

            for i in range(1,len(sap_bench_gab_df.columns)):
                # pick kolom satu-satu
                df_for_arima = sap_bench_gab_df.loc[:,['date',sap_bench_gab_df.columns[i]]].set_index('date')
                df_for_arima

                # split data jadi 3 bulan kebelakang dan kedepan
                train = df_for_arima.iloc[:-2]
                test = df_for_arima.iloc[-2:]

                # model untuk mencari mape
                arima_model =  auto_arima(train,start_p=0, d=1, start_q=0, 
                                          max_p=5, max_d=5, max_q=5, start_P=0, 
                                          D=1, start_Q=0, max_P=5, max_D=5,
                                          max_Q=5, m=12, seasonal=True, 
                                          error_action='warn',trace = True,
                                          supress_warnings=True,stepwise = True,
                                          random_state=20,n_fits = 50 )

                prediction = pd.DataFrame(arima_model.predict(n_periods = 2), index=test.index)
                prediction.columns = ['predicted_{}'.format(sap_bench_gab_df.columns[i])]
                prediction['actual_{}'.format(sap_bench_gab_df.columns[i])] = test

                # cari mape (error)
                mape = mean_absolute_percentage_error(prediction['actual_{}'.format(sap_bench_gab_df.columns[i])], prediction['predicted_{}'.format(sap_bench_gab_df.columns[i])])
                prediction['MAPE_{}'.format(sap_bench_gab_df.columns[i])] = mape

                # model # buat nambah tanggal prediksi kedepan
                arima_model =  auto_arima(df_for_arima,start_p=0, d=1, start_q=0, 
                                          max_p=5, max_d=5, max_q=5, start_P=0, 
                                          D=1, start_Q=0, max_P=5, max_D=5,
                                          max_Q=5, m=12, seasonal=True, 
                                          error_action='warn',trace = True,
                                          supress_warnings=True,stepwise = True,
                                          random_state=20,n_fits = 50 )

                pred_next_3_month = pd.DataFrame(arima_model.predict(n_periods = 10), index=pd.date_range(start=prediction.index.to_series().astype(str)[-1], periods=11, freq ='M')[1:].to_period('M'))
                pred_next_3_month = pred_next_3_month.reset_index().rename(columns={'index':'date', 0:'predicted_{}'.format(sap_bench_gab_df.columns[i])})
                pred_next_3_month

                # df gab train with test predict for 3 month
                df_gab_pred_next_3_month = pd.concat([prediction.reset_index(), pred_next_3_month])
                df_gab_pred_next_3_month['MAPE_{}'.format(sap_bench_gab_df.columns[i])] = df_gab_pred_next_3_month['MAPE_{}'.format(sap_bench_gab_df.columns[i])].fillna(mape)

                #df final gab all data
                df_gab_pred_next_3_month_final = pd.concat([train.rename(columns={'{}'.format(sap_bench_gab_df.columns[i]) : 'actual_{}'.format(sap_bench_gab_df.columns[i])}).reset_index(), df_gab_pred_next_3_month])
                df_gab_pred_next_3_month_final['MAPE_{}'.format(sap_bench_gab_df.columns[i])] = df_gab_pred_next_3_month_final['MAPE_{}'.format(sap_bench_gab_df.columns[i])].fillna(mape)
                # df_gab_pred_next_3_month_final['Material Number'] = etn

                #gabungin keseluruhan data per ematn (bench+sap)
                if i == 1 :
                    df_final = pd.DataFrame()
                    df_final = df_gab_pred_next_3_month_final
                else :
                    df_final = df_final.merge(df_gab_pred_next_3_month_final, on='date', how='left')
                    df_final['Material Number'] = j


            df_final_all_ematn = pd.concat([df_final_all_ematn, df_final])
            
        #=============================================
        # KCL
        if j in ['3000018','3000072','3000048','3000019']:
            # atur usd df
            df_usd_idr_1 = df_usd_idr.loc[:,['Tanggal','Terakhir']]
            df_usd_idr_1['Tanggal'] = pd.to_datetime(df_usd_idr['Tanggal'])
            df_usd_idr_1['Terakhir'] = df_usd_idr_1['Terakhir'].str[:-2].str.replace('.','').astype(float)
            df_usd_idr_1_for_corr = df_usd_idr_1.groupby(pd.Grouper(key='Tanggal', axis=0, 
                                    freq='Y')).mean().to_period('Y').reset_index().rename(columns={'Tanggal':'date'})

            # atur idr to usd
            df_for_corr = df_proc_2.reset_index().groupby(pd.Grouper(key='date', axis=0, 
                                    freq='Y')).mean().to_period('Y').reset_index()

            df_for_corr_tousd = df_for_corr.merge(df_usd_idr_1_for_corr, on='date', how='left')
            df_for_corr_tousd['UNITPRICE_USD'] = df_for_corr_tousd['UNITPRICE'] / df_for_corr_tousd['Terakhir']
            df_for_corr_tousd['date'] = df_for_corr_tousd['date'].astype(str).str[:4]
            df_for_corr_tousd.drop(columns=['UNITPRICE','Terakhir'],inplace=True)

            # CORRELATION
            # get 5 name good corr
            df_bench_kcl_new = df_bench_kcl.copy()
            df_for_corr_tousd['date'] = pd.to_datetime(df_for_corr_tousd['date'], format='%Y').dt.to_period('Y')
            df_bench_kcl_new['date'] = pd.to_datetime(df_bench_kcl_new['date'], format='%Y').dt.to_period('Y')
            name_country_list = df_for_corr_tousd.merge(df_bench_kcl_new, on='date', how='left').dropna().corr()['UNITPRICE_USD'].sort_values(ascending=False).index[1:6].to_list()

            # # ninu

            # make dataframe from all good correlation with data
            df_country_list = df_bench_kcl_new.loc[:,[name_country_list[0]]]
            df_country_list['date'] = pd.to_datetime(df_bench_kcl_new['date'].astype(str).str[:4] + '-' + df_bench_kcl_new['date'].astype(str).str[5:])
            df_country_list = df_country_list.set_index('date').to_period('Y').reset_index()

            # merge two tables sap & bench
            sap_bench_gab_df = df_country_list.merge(df_proc_2.reset_index().groupby(pd.Grouper(key='date', axis=0, 
                                    freq='Y')).mean().to_period('Y').reset_index(), on='date', how='left').dropna()

            sap_bench_gab_df = sap_bench_gab_df.merge(df_usd_idr_1_for_corr, on='date', how='left')
            sap_bench_gab_df['UNITPRICE_USD'] = sap_bench_gab_df['UNITPRICE'] / sap_bench_gab_df['Terakhir']
            sap_bench_gab_df.drop(columns=['UNITPRICE','Terakhir'],inplace = True)

            for i in range(1,len(sap_bench_gab_df.columns)):
                # pick kolom satu-satu
                df_for_arima = sap_bench_gab_df.loc[:,['date',sap_bench_gab_df.columns[i]]].set_index('date')
                df_for_arima

                # split data jadi 3 bulan kebelakang dan kedepan
                train = df_for_arima.iloc[:-2]
                test = df_for_arima.iloc[-2:]

                # model untuk mencari mape
                arima_model =  auto_arima(train,start_p=0, d=1, start_q=0, 
                                          max_p=5, max_d=5, max_q=5, start_P=0, 
                                          D=1, start_Q=0, max_P=5, max_D=5,
                                          max_Q=5, seasonal=True, 
                                          error_action='warn',trace = True,
                                          supress_warnings=True,stepwise = True,
                                          random_state=20,n_fits = 50 )

                prediction = pd.DataFrame(arima_model.predict(n_periods = 2), index=test.index)
                prediction.columns = ['predicted_{}'.format(sap_bench_gab_df.columns[i])]
                prediction['actual_{}'.format(sap_bench_gab_df.columns[i])] = test

                # cari mape (error)
                mape = mean_absolute_percentage_error(prediction['actual_{}'.format(sap_bench_gab_df.columns[i])], prediction['predicted_{}'.format(sap_bench_gab_df.columns[i])])
                prediction['MAPE_{}'.format(sap_bench_gab_df.columns[i])] = mape

                # model # buat nambah tanggal prediksi kedepan
                arima_model =  auto_arima(df_for_arima,start_p=0, d=1, start_q=0, 
                                          max_p=5, max_d=5, max_q=5, start_P=0, 
                                          D=1, start_Q=0, max_P=5, max_D=5,
                                          max_Q=5, seasonal=True, 
                                          error_action='warn',trace = True,
                                          supress_warnings=True,stepwise = True,
                                          random_state=20,n_fits = 50 )

                pred_next_3_month = pd.DataFrame(arima_model.predict(n_periods = 3), index=pd.date_range(start=prediction.index.to_series().astype(str)[-1], periods=4, freq ='Y')[1:].to_period('Y'))
                pred_next_3_month = pred_next_3_month.reset_index().rename(columns={'index':'date', 0:'predicted_{}'.format(sap_bench_gab_df.columns[i])})
                pred_next_3_month


                # df gab train with test predict for 3 month
                df_gab_pred_next_3_month = pd.concat([prediction.reset_index(), pred_next_3_month])
                df_gab_pred_next_3_month['MAPE_{}'.format(sap_bench_gab_df.columns[i])] = df_gab_pred_next_3_month['MAPE_{}'.format(sap_bench_gab_df.columns[i])].fillna(mape)

                #df final gab all data
                df_gab_pred_next_3_month_final = pd.concat([train.rename(columns={'{}'.format(sap_bench_gab_df.columns[i]) : 'actual_{}'.format(sap_bench_gab_df.columns[i])}).reset_index(), df_gab_pred_next_3_month])
                df_gab_pred_next_3_month_final['MAPE_{}'.format(sap_bench_gab_df.columns[i])] = df_gab_pred_next_3_month_final['MAPE_{}'.format(sap_bench_gab_df.columns[i])].fillna(mape)
                # df_gab_pred_next_3_month_final['Material Number'] = etn

                #gabungin keseluruhan data per ematn (bench+sap)
                if i == 1 :
                    df_final = pd.DataFrame()
                    df_final = df_gab_pred_next_3_month_final
                else :
                    df_final = df_final.merge(df_gab_pred_next_3_month_final, on='date', how='left')
                    df_final['Material Number'] = j


            df_final_all_ematn = pd.concat([df_final_all_ematn, df_final])
            
    
    # untuk memberikan legend dan menandakan mana nilai actual & prediction
    
    df_final_all_ematn.loc[(df_final_all_ematn["actual_UNITPRICE_USD"] != np.nan) & (df_final_all_ematn["predicted_UNITPRICE_USD"]!= np.nan), "type_price"] = 'Actual / Predicted Price'
    df_final_all_ematn.loc[(df_final_all_ematn["actual_UNITPRICE_USD"] != np.nan) & (df_final_all_ematn["predicted_UNITPRICE_USD"].isna()), "type_price"] = 'Actual Price'
    df_final_all_ematn.loc[(df_final_all_ematn["actual_UNITPRICE_USD"].isna()) & (df_final_all_ematn["predicted_UNITPRICE_USD"] != np.nan), "type_price"] = 'Predicted Price'
            
    return df_final_all_ematn

In [102]:
result = pred_using_arima_for_dapkcl(DAPKCL_EMATN)
result

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=192.382, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=185.290, Time=0.05 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=184.419, Time=0.06 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=183.421, Time=0.03 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=184.420, Time=0.06 sec
 ARIMA(0,1,1)(1,1,1)[12]             : AIC=186.419, Time=0.30 sec
 ARIMA(1,1,1)(0,1,0)[12]             : AIC=185.174, Time=0.07 sec
 ARIMA(0,1,2)(0,1,0)[12]             : AIC=185.364, Time=0.13 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=183.292, Time=0.03 sec
 ARIMA(1,1,0)(0,1,1)[12]             : AIC=185.290, Time=0.06 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=187.290, Time=0.11 sec
 ARIMA(2,1,0)(0,1,0)[12]             : AIC=185.278, Time=0.05 sec
 ARIMA(2,1,1)(0,1,0)[12]             : AIC=inf, Time=0.16 sec
 ARIMA(1,1,0)(0,1,0)[12] intercept   : AIC=181.098, Time=0.06 sec
 ARIMA(1,1,0)(1,1,0)[12] intercept   

 ARIMA(1,1,2)(0,1,0)[12]             : AIC=inf, Time=0.14 sec
 ARIMA(0,1,1)(0,1,0)[12] intercept   : AIC=212.872, Time=0.04 sec
 ARIMA(0,1,1)(1,1,0)[12] intercept   : AIC=210.716, Time=0.09 sec
 ARIMA(0,1,1)(2,1,0)[12] intercept   : AIC=212.717, Time=0.20 sec
 ARIMA(0,1,1)(1,1,1)[12] intercept   : AIC=inf, Time=0.17 sec
 ARIMA(0,1,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(0,1,1)(2,1,1)[12] intercept   : AIC=214.716, Time=0.23 sec
 ARIMA(0,1,0)(1,1,0)[12] intercept   : AIC=212.481, Time=0.11 sec
 ARIMA(1,1,1)(1,1,0)[12] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(0,1,2)(1,1,0)[12] intercept   : AIC=209.316, Time=0.21 sec
 ARIMA(0,1,2)(0,1,0)[12] intercept   : AIC=inf, Time=0.11 sec
 ARIMA(0,1,2)(2,1,0)[12] intercept   : AIC=211.315, Time=0.42 sec
 ARIMA(0,1,2)(1,1,1)[12] intercept   : AIC=211.317, Time=0.32 sec
 ARIMA(0,1,2)(0,1,1)[12] intercept   : AIC=inf, Time=0.29 sec
 ARIMA(0,1,2)(2,1,1)[12] intercept   : AIC=213.316, Time=0.35 sec
 ARIMA(1,1,2)(1,1,0)[12] intercept

 ARIMA(3,1,0)(2,1,2)[12]             : AIC=234.291, Time=1.27 sec
 ARIMA(4,1,0)(1,1,1)[12]             : AIC=238.078, Time=0.58 sec
 ARIMA(3,1,1)(1,1,1)[12]             : AIC=inf, Time=0.50 sec
 ARIMA(2,1,1)(1,1,1)[12]             : AIC=inf, Time=0.50 sec
 ARIMA(4,1,1)(1,1,1)[12]             : AIC=inf, Time=0.55 sec
 ARIMA(3,1,0)(1,1,1)[12] intercept   : AIC=225.143, Time=0.52 sec
 ARIMA(3,1,0)(0,1,1)[12] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(3,1,0)(1,1,0)[12] intercept   : AIC=inf, Time=0.42 sec
 ARIMA(3,1,0)(2,1,1)[12] intercept   : AIC=inf, Time=1.18 sec
 ARIMA(3,1,0)(1,1,2)[12] intercept   : AIC=inf, Time=0.99 sec
 ARIMA(3,1,0)(0,1,0)[12] intercept   : AIC=274.214, Time=0.12 sec
 ARIMA(3,1,0)(0,1,2)[12] intercept   : AIC=inf, Time=0.76 sec
 ARIMA(3,1,0)(2,1,0)[12] intercept   : AIC=inf, Time=0.92 sec
 ARIMA(3,1,0)(2,1,2)[12] intercept   : AIC=inf, Time=1.50 sec
 ARIMA(2,1,0)(1,1,1)[12] intercept   : AIC=inf, Time=0.54 sec
 ARIMA(4,1,0)(1,1,1)[12] intercept   : AIC=inf, Time=0

 ARIMA(0,1,1)(0,1,1)[12]             : AIC=104.880, Time=0.14 sec
 ARIMA(0,1,0)(1,1,0)[12]             : AIC=inf, Time=0.17 sec
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=105.281, Time=0.03 sec
 ARIMA(0,1,0)(1,1,1)[12]             : AIC=107.281, Time=0.03 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=104.318, Time=0.04 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=inf, Time=0.03 sec
 ARIMA(1,1,1)(0,1,0)[12]             : AIC=inf, Time=0.05 sec
 ARIMA(0,1,0)(0,1,0)[12] intercept   : AIC=105.281, Time=0.01 sec

Best model:  ARIMA(0,1,0)(0,1,0)[12]          
Total fit time: 0.694 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=126.746, Time=0.02 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=inf, Time=0.20 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=0.04 sec
 ARIMA(0,1,0)(1,1,0)[12]             : AIC=128.746, Time=0.03 sec
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=128.746, Time=0.05 sec
 ARIMA(0,1,0)(1,1,1)[12]             : AI

 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=54.715, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=53.638, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.07 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 0.177 seconds


,date,actual_MAP fob Baltic_max,predicted_MAP fob Baltic_max,MAPE_MAP fob Baltic_max,actual_UNITPRICE_USD,predicted_UNITPRICE_USD,MAPE_UNITPRICE_USD,Material Number,actual_DAP fob China_max,predicted_DAP fob China_max,MAPE_DAP fob China_max,actual_Potash MOP (Standard) fob Baltic/Black Sea,predicted_Potash MOP (Standard) fob Baltic/Black Sea,MAPE_Potash MOP (Standard) fob Baltic/Black Sea,type_price
0,2019-01,415.0,NaN,0.041761,416.577667,NaN,0.049592,1000017,NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
1,2019-02,410.0,NaN,0.041761,395.502896,NaN,0.049592,1000017,NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
2,2019-03,400.0,NaN,0.041761,414.880678,NaN,0.049592,1000017,NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
3,2019-04,380.0,NaN,0.041761,399.118212,NaN,0.049592,1000017,NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
4,2019-05,373.0,NaN,0.041761,394.932491,NaN,0.049592,1000017,NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2020,NaN,NaN,NaN,137.157942,21.443648,0.875009,3000019,NaN,NaN,NaN,211.0,296.001666,0.313248,Actual / Predicted Price
4,2021,NaN,NaN,NaN,421.724634,39.489898,0.875009,3000019,NaN,NaN,NaN,389.0,302.002396,0.313248,Actual / Predicted Price
5,2022,NaN,NaN,NaN,NaN,526.209637,0.875009,3000019,NaN,NaN,NaN,NaN,389.000000,0.313248,Predicted Price
6,2023,NaN,NaN,NaN,NaN,630.694639,0.875009,3000019,NaN,NaN,NaN,NaN,389.000000,0.313248,Predicted Price


In [104]:
result.loc[result['Material Number'] == "1000017", "Material Number"] = "1000017 : DAP 64 (PKG)"
result.loc[result['Material Number'] == "3000018", "Material Number"] = "3000018 : KCL Kanada - Laos (PKG, PKC, PKT, PSP)"
result.loc[result['Material Number'] == "3000019", "Material Number"] = "3000019 : KCL Flake (PKT)"
result.loc[result['Material Number'] == "3000048", "Material Number"] = "3000048 : KCL Laos (PKC)"
result.loc[result['Material Number'] == "3000072", "Material Number"] = "3000072 : KCL Putih (PKG)"
result.loc[result['Material Number'] == "3000266", "Material Number"] = "3000266 : DAP 61 (PKC, PKT)"
result.loc[result['Material Number'] == "3000499", "Material Number"] = "3000499 : DAP 64 (PKC, PKT, PSP)"
result.loc[result['Material Number'] == "3000539", "Material Number"] = "3000539 : DAP 61 (PSP)"

In [103]:
# result.to_csv('export_untuk_db_100622.csv', index=False)

In [105]:
result

,date,actual_MAP fob Baltic_max,predicted_MAP fob Baltic_max,MAPE_MAP fob Baltic_max,actual_UNITPRICE_USD,predicted_UNITPRICE_USD,MAPE_UNITPRICE_USD,Material Number,actual_DAP fob China_max,predicted_DAP fob China_max,MAPE_DAP fob China_max,actual_Potash MOP (Standard) fob Baltic/Black Sea,predicted_Potash MOP (Standard) fob Baltic/Black Sea,MAPE_Potash MOP (Standard) fob Baltic/Black Sea,type_price
0,2019-01,415.0,NaN,0.041761,416.577667,NaN,0.049592,1000017 : DAP 64 (PKG),NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
1,2019-02,410.0,NaN,0.041761,395.502896,NaN,0.049592,1000017 : DAP 64 (PKG),NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
2,2019-03,400.0,NaN,0.041761,414.880678,NaN,0.049592,1000017 : DAP 64 (PKG),NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
3,2019-04,380.0,NaN,0.041761,399.118212,NaN,0.049592,1000017 : DAP 64 (PKG),NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
4,2019-05,373.0,NaN,0.041761,394.932491,NaN,0.049592,1000017 : DAP 64 (PKG),NaN,NaN,NaN,NaN,NaN,NaN,Actual Price
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2020,NaN,NaN,NaN,137.157942,21.443648,0.875009,3000019 : KCL Flake (PKT),NaN,NaN,NaN,211.0,296.001666,0.313248,Actual / Predicted Price
4,2021,NaN,NaN,NaN,421.724634,39.489898,0.875009,3000019 : KCL Flake (PKT),NaN,NaN,NaN,389.0,302.002396,0.313248,Actual / Predicted Price
5,2022,NaN,NaN,NaN,NaN,526.209637,0.875009,3000019 : KCL Flake (PKT),NaN,NaN,NaN,NaN,389.000000,0.313248,Predicted Price
6,2023,NaN,NaN,NaN,NaN,630.694639,0.875009,3000019 : KCL Flake (PKT),NaN,NaN,NaN,NaN,389.000000,0.313248,Predicted Price


# EXPORT TO DATABASE

In [106]:
df_final_all_ematn_db = result.replace(np.nan, '', regex=True)

# DELETE TABLE AND UPDATE EXPORT DATA TO DB
import pymysql
mysql_connection = pymysql.connect(host='localhost',
                    user='root',
                    password='',
                    db='csv_db 6',
                    charset='utf8',
                    cursorclass=pymysql.cursors.DictCursor)
                    
cursor = mysql_connection.cursor()

sql = "DELETE FROM `price_pred_dapkcl_1`"
cursor.execute(sql)

# creating column list for insertion
cols = "`,`".join([str(i) for i in df_final_all_ematn_db.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_final_all_ematn_db.iterrows():

    sql = "INSERT INTO `price_pred_dapkcl_1` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    
    # the connection is not autocommitted by default, so we must commit to save our changes
    mysql_connection.commit()

# CORET-CORETAN

In [29]:
# read data
df_proc_1 = df_all.copy()
df_proc_1

# benerin ematn (ilangin 0 nya)
df_proc_1['EMATN'] = df_proc_1['EMATN'].astype(str)
df_proc_1['EMATN'] = df_proc_1['EMATN'].str[-7:]

# pick ematn
etn = '1000017'
df_proc_1 = df_proc_1[df_proc_1['EMATN'] == etn].copy()

# benerin bedat
df_proc_1['BEDAT'] = df_proc_1['BEDAT'].astype(str)
df_proc_1['BEDAT'] = df_proc_1['BEDAT'].str[6:8] + "/" + df_proc_1['BEDAT'].str[4:6] + "/" + df_proc_1['BEDAT'].str[:4]
df_proc_1['BEDAT'] = pd.to_datetime(df_proc_1['BEDAT'])

# benerin currency
df_proc_1['CURRENCY_RATE'] = df_proc_1['CURRENCY_RATE'].astype(float) * 1000
df_proc_1['UNITPRICE'] = df_proc_1['UNITPRICE'].astype(float)
df_proc_1.loc[df_proc_1["WAERS"] == "USD", "UNITPRICE"] = df_proc_1['UNITPRICE']*df_proc_1['CURRENCY_RATE']
df_proc_1 = df_proc_1[df_proc_1['EMATN'] == etn].loc[:,['BEDAT','UNITPRICE']].sort_values('BEDAT')
df_proc_1 = df_proc_1.rename(columns = {'BEDAT' : 'date'})

# per-week mode
df_proc_2 = df_proc_1.groupby(pd.Grouper(key='date', axis=0, 
                        freq='W')).apply(lambda x: x.mode())
df_proc_2 = df_proc_2.drop(columns=('date')).reset_index().drop(columns=('level_1')).groupby(pd.Grouper(key='date', axis=0, 
                        freq='W')).mean()

# interpolate mengisi data yang nan/kosong
df_proc_2 = df_proc_2.interpolate(method='linear', axis=0)

# atur usd df
df_usd_idr_1 = df_usd_idr.loc[:,['Tanggal','Terakhir']]
df_usd_idr_1['Tanggal'] = pd.to_datetime(df_usd_idr['Tanggal'])
df_usd_idr_1['Terakhir'] = df_usd_idr_1['Terakhir'].str[:-2].str.replace('.','').astype(float)
df_usd_idr_1_for_corr = df_usd_idr_1.groupby(pd.Grouper(key='Tanggal', axis=0, 
                        freq='M')).mean().to_period('M').reset_index().rename(columns={'Tanggal':'date'})

# atur idr to usd
df_for_corr = df_proc_2.reset_index().groupby(pd.Grouper(key='date', axis=0, 
                        freq='M')).mean().to_period('M').reset_index()

df_for_corr_tousd = df_for_corr.merge(df_usd_idr_1_for_corr, on='date', how='left')
df_for_corr_tousd['UNITPRICE_USD'] = df_for_corr_tousd['UNITPRICE'] / df_for_corr_tousd['Terakhir']
df_for_corr_tousd['date'] = df_for_corr_tousd['date'].astype(str).str[:4] + 'M' + df_for_corr_tousd['date'].astype(str).str[5:7]
df_for_corr_tousd.drop(columns=['UNITPRICE','Terakhir'],inplace=True)


 # CORRELATION
# get 5 name good corr
name_country_list = df_for_corr_tousd.merge(df_bench_dap, on='date', how='left').dropna().corr()['UNITPRICE_USD'].sort_values(ascending=False).index[1:6].to_list()

# make dataframe from all good correlation with data
df_country_list = df_bench_dap.loc[:,[name_country_list[0]]]
df_country_list['date'] = pd.to_datetime(df_bench_dap['date'].str[:4] + '-' + df_bench_dap['date'].str[5:])
df_country_list = df_country_list.set_index('date').to_period('M').reset_index()

# merge two tables sap & bench
sap_bench_gab_df = df_country_list.merge(df_proc_2.reset_index().groupby(pd.Grouper(key='date', axis=0, 
                        freq='M')).mean().to_period('M').reset_index(), on='date', how='left').dropna()

sap_bench_gab_df = sap_bench_gab_df.merge(df_usd_idr_1_for_corr, on='date', how='left')
sap_bench_gab_df['UNITPRICE_USD'] = sap_bench_gab_df['UNITPRICE'] / sap_bench_gab_df['Terakhir']
sap_bench_gab_df.drop(columns=['UNITPRICE','Terakhir'],inplace = True)

In [33]:
# sap_bench_gab_df.to_excel("data_sapfertecon_MaterialNumber1000017.xlsx",index=False)